# Postprocessing of simulation data from Kinetiscope
This jupyter notebook demonstrates how to use the StackSim.py module to extract simulation data from XML files generated by the kinetic Monte Carlo simulator [Kinetiscope](http://www.kinetiscope.com). These simulations use a set of chemical and diffusion equations to simulate the oxidation of organic aerosol initiated at the surface. The simulations are set up as a stack of thin rectangular prisms that represent spherical shells in an organic aerosol particle. Because of this change in geometry, the data from the simulations has to be postprocessed so that the outer rectangular prisms are weighted more than the inner rectangular prisms. Here, the height of each prism is taken as the thickness of the spherical shell. The following figure illustrates a schematic of the simulation: ![simulation schematic](overview graphic.png)

Note: If you do not wish to see the code and just care about the results, feel free to press the "Hide Code" button below to hide the code used to postprocess and plot the data.

In [1]:
%%HTML 
<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>

In [21]:
# Import modules needed for postprocessing and unzip and load data

# Contains classes for compartments and simulations
import StackSim


import warnings
warnings.filterwarnings('ignore')
import os
import requests
import zipfile
import errno
import numpy as np
import pandas as pd
from matplotlib import ticker
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import re
import seaborn as sns

# Create subdirectory for data if it does not exist

subdir = os.path.join(os.path.curdir, "simdata")

if not os.path.isdir(subdir):
    os.mkdir(subdir)
 

# Unzip XML files (.zip file comes with github repository)

with zipfile.ZipFile("model_scenarios_xml.zip", "r") as zip_ref:
    zip_ref.extractall(subdir)
    
# Create scenario-file dictionary

ScenarioFiles = {'Scenario 1' : 'tri_109nm_sc1.xml', 
                 'Scenario 1 decay' : 'tri_145nm_sc1.xml',
                 'Scenario 1A' : 'tri_109nm_sc1a.xml', 
                 'Scenario 1A decay' : 'tri_145nm_sc1a.xml',
                 'Scenario 2' : 'tri_109nm_sc2.xml', 
                 'Scenario 2 decay' : 'tri_145nm_sc2.xml',
                 'Scenario 2A' : 'tri_109nm_sc2a.xml',
                 'Scenario 2A decay' : 'tri_145nm_sc2a.xml'}

# Create empty scenario dictionary

Scenarios = dict()

## Weighting of functional groups and carbon backbones
After the loading the data, the species amounts are weighted by their relative contributions to different elements and mass. In the simulation, a molecule is represented as a collection of oxygen and hydrogen functional groups and carbon number backbones. For example, a secondary alcohol group (OHCH_sec) counts once for oxygen and twice for hydrogen, and a 18-carbon fatty acid counts for 18 carbon atoms. These weights are used to find the average elemental composition and mass of the organic aerosol.

In [22]:
# Generate lists for finding weighting aggregate species from individual
# functional group or carbon backbone species

carbon_min, carbon_max = 1, 30

missing_carbon_no = [1, 30]

# Create carbon-30 lumped species from each component

carbon_no_dict = {"nC30" : [["C30", 1], ["C30_COOH", 1], ["C30_COOH_O", 1], ["C30_HOOCCOOH", 1], ["C30_O2", 1], ["C30_O", 1]]}

# Initialize list of lists

carbon_list = [["nC30", 30]]

for i in range(carbon_min, carbon_max+1, 1):
    if i in missing_carbon_no:
        pass
    else:
        no_C = "C"+str(i)
        
        # Add entry to lumped carbon number dictionary
        carbon_no_dict["n"+no_C] = [[no_C, 1], [no_C+"_O2", 1], [no_C+"_COOH", 1], [no_C+"_COOH_O", 1], [no_C+"_HOOCCOOH", 1]]
        
        # Add entry to list of list for carbon number weighting
        carbon_list.append(["n"+no_C, i])

# Generate oxygen and hydrogen species weighting lists

oxygen_list = [["OC_sec", 1], ["OCH_prim", 1], ["OHCH2_prim", 1], ["OHCH_sec", 1],
               ["OC_alpha", 1], ["OHCH_alpha", 1], ["HO_OOC_prim", 3],
               ["HOOCH2_prim", 2], ["HOOCH_alpha", 2], ["HOOCH_sec", 2], ["HOOC_prim", 2]]

hydrogen_list = [["CH3_prim", 3], ["CH3_prim_s", 3], ["CH2_sec", 2], ["CH2_alpha", 2], 
                 ["OCH_prim", 1], ["OHCH_sec", 2], ["OHCH2_prim", 3], ["OHCH_alpha", 2],
                 ["HO_OOC_prim", 1], ["HOOCH2_prim", 3], ["HOOCH_alpha", 2], ["HOOCH_sec", 2], ["HOOC_prim", 1]]

# Generate mass weighting lists

mass_list = [["carbon", 12], ["oxygen", 16], ["hydrogen", 1]]

mass_list_r = [["carbon_r", 12], ["oxygen_r", 16], ["hydrogen_r", 1]]

## Finding aggregate species and radial correction

After initializing the weighting lists for each species, the simulation files are processed using the Simulation and Compartment classes in StackSim.py. First, the simulation results are loaded into simulation and compartment objects from the XML files. Then, aggregate species for carbon number, total carbon; oxygen; and hydrogen, and mass are created. The species ratios H/C and O/C are also calculated from these values.

Radial corrections are also calculated for each of these species and the starting material to properly weight the outer compartments. The thickness of each compartment is considered to be the same as the thickness of an equivalent spherical shell. The initial radial $r_i$ and final radial coordinates $r_f$ are then used to find the volume of a spherical shell ($V_{ \text{shell} }$) as follows:

$V_{ \text{shell} } = 4/3 \pi (r^3_f - r^3_i)$.

The ratio of the volume of the shell and the volume of the compartment ($V_{ \text{box} }$) is then used to weight the amount of any given species ($N$) by its position along the radial axis as follows:

$N_{ \text{corr} } = V_{ \text{shell} } N / V_{ \text{box} }$.

Note: the below code can take a while to run so please be patient since it is loading and processing 8 large files.

In [23]:
for scenario in ScenarioFiles.keys():
    
    # Load and process simulation data from simulation    
    Scenarios[scenario] = StackSim.SimulationData(os.path.join(subdir, ScenarioFiles[scenario]))
    
    # Calculate aggregate carbon species and radial correction
    for carbon_no in carbon_no_dict.keys():
        Scenarios[scenario].calcAggregateSpecies(carbon_no, carbon_no_dict[carbon_no])
        Scenarios[scenario].calcRadialCorrection(carbon_no, reverse_axis=True)
    

    # Calculate each element and mass    
    Scenarios[scenario].calcAggregateSpecies("carbon", carbon_list)

    Scenarios[scenario].calcAggregateSpecies("oxygen", oxygen_list)

    Scenarios[scenario].calcAggregateSpecies("hydrogen", hydrogen_list)     
    
    Scenarios[scenario].calcAggregateSpecies("mass", mass_list)
    
    # Calculate radial correction for each element and mass    
    
    Scenarios[scenario].calcRadialCorrection("mass", reverse_axis=True)
    Scenarios[scenario].calcRadialCorrection("oxygen", reverse_axis=True)
    Scenarios[scenario].calcRadialCorrection("carbon", reverse_axis=True)
    Scenarios[scenario].calcRadialCorrection("hydrogen", reverse_axis=True)
    
    # Calculate radial correction for triacontane    
    
    Scenarios[scenario].calcRadialCorrection("Tri", reverse_axis=True)
    
    # Calculate mass of aerosol    
    
    Scenarios[scenario].calcAggregateSpecies("mass_r", mass_list_r)
    
    Scenarios[scenario].calcSpeciesRatio("O/C ratio", "oxygen_r", "carbon_r")
    Scenarios[scenario].calcSpeciesRatio("H/C ratio", "hydrogen_r", "carbon_r")

## Summary of each scenario

After processing the data, the simulation data for each scenario are then collected into data frames. A panel containing contour maps showing the internal distribution of the starting material, O/C ratio, and peroxy radicals are also generated. These data frames are used to summarize and plot several features of the simulations. The resulting data are also saved into an Excel workbook for each simulation.

In [25]:
%matplotlib inline

# OH and concentration and units for OH exposure
OH_conc = 2.5E11
OH_exp_units = r'OH exposure (molecules cm$^{-3}$ s)'

for scenario in ScenarioFiles.keys():
    # Create reference to current simulation to save on typing
    simulation = Scenarios[scenario]
    
    # Print basic scenario information
    print(re.sub(" decay", "", scenario))
    
    # Find initial radius of particle
    last_cmpt = simulation.compartments[(0, simulation.num_rows-1, 0)]
    radius = last_cmpt.positions["y"]+last_compt.dimensions["y"]
    
    # Print particle diameter
    print(str(2*radius))
    

Scenario 1
[  1.08000000e-05   1.07928251e-05   1.07844176e-05   1.07753030e-05
   1.07691310e-05   1.07623542e-05   1.07558139e-05   1.07478880e-05
   1.07253773e-05   1.06909929e-05   1.06850003e-05   1.06796298e-05
   1.06723086e-05   1.06648600e-05   1.06564188e-05   1.06439763e-05
   1.06288232e-05   1.05935362e-05   1.05843615e-05   1.05781795e-05
   1.05715502e-05   1.05639509e-05   1.05564203e-05   1.05355977e-05
   1.04970680e-05   1.04890588e-05   1.04815068e-05   1.04755538e-05
   1.04676542e-05   1.04596451e-05   1.04433560e-05   1.04048827e-05
   1.03957184e-05   1.03881744e-05   1.03839938e-05   1.03757623e-05
   1.03653453e-05   1.03466198e-05   1.03104481e-05   1.02972076e-05
   1.02937611e-05   1.02874804e-05   1.02795138e-05   1.02653932e-05
   1.02468673e-05   1.02125708e-05   1.02076710e-05   1.02020236e-05
   1.01956067e-05   1.01908092e-05   1.01817447e-05   1.01651000e-05
   1.01218908e-05   1.01132304e-05   1.01091210e-05   1.01050512e-05
   1.01025382e-05   1.0